# Installation and Repository cloning

In [1]:
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
#!pip install adversarial-robustness-toolbox -U
#!pip install git+https://github.com/RobustBench/robustbench.git
#!pip install multiprocess
#!pip install matplotlib
#!pip install pillow
#!pip install foolbox
#!pip install numba

## Import Libraries

In [1]:
import torch
import utils
import adversarial_attack.attack_utils as attack_utils

device = 'cpu' #torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/shao/anaconda3/envs/art/lib/python3.12/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/home/shao/anaconda3/envs/art/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/shao/anaconda3/envs/art/lib/python3.12/site-packages/timm/models/helpers.py:7: FutureWarning: Importing from timm.models.helpers is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


## Load and Prepare Adversarial Model

In [2]:
model = 'CroceL1' # MainiAVG, MainiMSD, CroceL1, Wang2023Better_WRN-28-10, Wang2023Better_WRN-28-10
net, art_net, fb_net, alias = utils.get_model(modelname=model, norm='L2') # specify the norm in case of robustbench models

no input normalization


/home/shao/elastic_net_adversarial_attack/utils.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load('./models/pretrained_models/CroceL1.pth')


## Load the dataset

In [3]:
splitsize = 100 # full, int: splitsize
xtest, ytest = utils.load_dataset(dataset_split=splitsize)

Files already downloaded and verified


## Test Accuracy

In [4]:
utils.test_accuracy(net, xtest, ytest)


Accuracy of the test set is: 84.000%



## Parameters 

In [5]:
Experiment = attack_utils.Experiment_class(art_net, fb_net, net, xtest, ytest, alias,
                        epsilon_l1 = 20, 
                        epsilon_l2= 0.5, 
                        eps_iter = 0.25,  
                        norm = 1,  # 1, 2, np.inf
                        max_iterations = 100, 
                        verbose=False)

# Hyperparameter Sweep Experiment

In [6]:
#hyperparameter sweep. Pick only one model.
attack_type = 'exp_attack'
hyperparameter_range=[0.01,0.1,1.0,10.0]
hyperparameter = 'learning_rate' #'beta'

results_dict_hyperparameter_sweep = Experiment.hyperparameter_sweep(hyperparameter=hyperparameter, range=hyperparameter_range, attack_type=attack_type)

		-------------- Hyperparameter Sweep for Attack: exp_attack: learning_rate = 0.01 ----------------



/home/shao/elastic_net_adversarial_attack/adversarial_attack/exp_attack.py:255: RuntimeWarning: invalid value encountered in log
  v_val = np.where(abc>=15.0,np.log(abc)-np.log(np.log(abc))+np.log(np.log(abc))/np.log(abc), lambertw( np.exp(abc), k=0).real)/b-a


KeyboardInterrupt: 

In [7]:
#Attack comparison
attack_types = [#'fast_gradient_method',
                #'projected_gradient_descent',
                #'pgd_early_stopping',
                #'auto_projected_gradient_descent',
                #'deep_fool',
                #'brendel_bethge',
                #'carlini_wagner_l2',
                'elastic_net',
                'exp_attack',
                #'exp_attack_smooth',
                #'exp_attack_l1',
                #'elastic_net_L1_rule',
                #'elastic_net_L1_rule_higher_beta',
                #'ART_AutoAttack',
                #'original_AutoAttack',
                #'original_AutoAttack_apgd-only'
                ]

results_dict_attack_comparison = Experiment.attack_comparison(attack_types)

		-------------------------- Processing Attack: exp_attack --------------------------

20 images done. Current Adversarial Accuracy (L1 / L2/ EN): 60.0 / 55.0/60.0%


KeyboardInterrupt: 